# Testes da rotina da classe das Equações de Mackey-Glass

## 1. Importando as bibliotecas necessárias

### Bibliotecas obrigatórias

In [1]:
import numpy as np
from scipy.integrate import odeint

%matplotlib widget
import matplotlib.pyplot as plt

import seaborn as sns # a biblioteca 'seaborn' contém vários estilos para os gráficos do 'matpĺotlib'

# agora, melhoramos a qualidade de saida e de visualizacao da imagem 
# alem de mudar a fonte padrao para uma do latex
sns.set_style("ticks")
plt.rcParams['savefig.dpi'] = 200
plt.rcParams["figure.dpi"] = 100

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Palatino"],
})

plt.style.use('dark_background')

### Bibliotecas não-obrigatórias

In [2]:
import sys 
sys.path.insert(0, '../../scripts')
import mackeyglassequations as mgeq

## 2. Definindo a função (debug)

Não é necessário executar essa linha se você estiver importando o arquivo com a classe.

In [2]:
class MackeyGlass:
    
    def __init__(self, P_inicial, gamma=0.1, beta=0.2, theta=1, tau=30, n=30):
        """
        Descrição:
        ----------
        Construtor da classe 'SistemaLorenz'

        Parâmetros:
        -----------
        estado_inicial: int ou float
            Concentração inicial dos glóbulos vermelhos   
        gamma: int ou float
            Parâmetro das Equações de Mackey-Glass
        beta: int ou float
            Parâmetro das Equações de Mackey-Glass
        theta: int ou float
            Parâmetro das Equações de Mackey-Glass
        tau: int
            Parâmetro das Equações de Mackey-Glass
        n: int
            Parâmetro das Equações de Mackey-Glass
            
        Retorna:
        --------
        Nada
        """
        
        if not (((type(P_inicial) is float) | (type(P_inicial) is int)) & (P_inicial > 0)):
            raise TypeError("A concentração inicial dos glóbulos vermelhos deve ser um int ou float maior que 0!") 
        
        if not (((type(gamma) is int) | (type(gamma) is float)) & (gamma > 0)):
            raise TypeError("Gamma deve ser um int ou float positivo!")

        if not (((type(beta) is int) | (type(beta) is float)) & (beta > 0)):
            raise TypeError("Beta deve ser um int ou float positivo!")

        if not (((type(theta) is int) | (type(theta) is float)) & (theta > 0)):
            raise TypeError("Theta deve ser um int ou float positivo!")
                
        if not (((type(tau) is int) | (type(tau) is float)) & (tau > 0)):
            raise TypeError("Tau deve ser um int ou float positivo!")
                
        if not ((type(n) is int) & (n > 0)):
            raise TypeError("n deve ser um int positivo!")
        
        self._estado_inicial = P_inicial
        self._gamma = gamma
        self._beta = beta
        self._theta = theta
        self._tau = tau
        self._n = n
        pass
    
    def _equacoes(self, estado_atual, t):
        """
        Descrição:
        ----------
        Função interna que retorna as equações do Sistema de Lorenz calculadas no instante t atual
        
        Parâmetros:
        -----------
        estado_atual: np.ndarray
            Vetor das posições xyz atuais do sistema    
        t: float
            Instante t no qual estamos calculando as derivadas
            
        Retorna:
        --------
        As equações de Mackey-Glass para os parâmetros estimados na forma de um array
        """
        
        if not (type(estado_atual) is np.ndarray):
            raise TypeError("O vetor estado atual deve ser um array do numpy!")        
        
        if not (type(t) is float):
            raise TypeError("t deve ser um float!")
        
        sigma = self._sigma
        beta = self._beta
        rho = self._rho
        x, y, z = estado_atual
        
        dx_dt = sigma * (y - x)
        dy_dt = x * (rho - z) - y
        dz_dt = x * y - beta * z
        return [dx_dt, dy_dt, dz_dt]
    
    def calcular(self, t_inicial, t_final, n_instantes=10000):
        """
        Descrição:
        ----------
        Evolui as equações de Mackey-Glass com base nas condições iniciais configuradas, para um t indo de t_inicial até t_final, com n_instantes calculados
        
        Parâmetros:
        -----------
        t_inicial: int
            Dia em que iniciamos os cálculos
        t_final: int
            Dia em que terminamos os cálculos
        n_instantes: int
            Número de instantes temporais em que faremos a estimação
            
        Retorna:
        --------
        Um vetor com as soluções estimadas e um vetor com os instantes temporais utilizados
        """
        
        if not ((type(t_inicial) is int) & (t_inicial >= 0)):
            raise TypeError("t_inicial deve ser um int não nulo!")
            
        if not ((type(t_final) is int) & (t_final > 0)):
            raise TypeError("t_final deve ser um int positivo!")
            
        if not ((type(n_instantes) is int) & (n_instantes > 0)):
            raise TypeError("n_instantes deve ser um int positivo!")            
        
        estado_inicial = self._estado_inicial
        instantes_temporais = np.linspace(t_inicial, t_final, n_instantes)
        
        solucoes = odeint(self._equacoes, t=instantes_temporais, y0=estado_inicial)
        return solucoes, instantes_temporais